In [1]:
import json
import networkx as nx
import pandas as pd

In [2]:
data = json.load(open("../data/GOOG_two_hop_subgraph.json"))

In [3]:
G = nx.Graph()

In [4]:
data[0]["nodes"][0]

{'identity': 1332,
 'labels': ['Company'],
 'properties': {'ticker': 'GOOG',
  'cik': '0001652044',
  'fiscalYearEnd': '1231',
  'businessSegments': '',
  'irsNumber': '611767919',
  'exchangeTicker': 'Nasdaq',
  'mailingAddress': '1600 AMPHITHEATRE PARKWAY MOUNTAIN VIEW CA 94043 CA',
  'sicCode': '7370',
  'stateCodeOfIncorporation': 'DE',
  'name': 'Alphabet Inc.',
  'productServices': '',
  'stateCode': 'CA',
  'businessAddress': '1600 AMPHITHEATRE PARKWAY MOUNTAIN VIEW CA 94043 CA 650-253-0000'},
 'elementId': '1332'}

In [5]:
company_nodes = set()

In [6]:
for i in data[0]["nodes"]:
    G.add_node(i["identity"], **i)
    
    if "Company" in i["labels"]:
        company_nodes.add(i["identity"])

In [7]:
unallowed_types = ["HAS_EXCHANGE_MARKET", "HAS_STATE_OF_INCORPORATION"]

In [8]:
data[0]["relationships"][0]

{'identity': 91665,
 'start': 1332,
 'end': 950,
 'type': 'HAS_STATE_LOCATION',
 'properties': {},
 'elementId': '91665',
 'startNodeElementId': '1332',
 'endNodeElementId': '950'}

In [10]:
for i in data[0]["relationships"]:
    if i["type"] not in unallowed_types:
        G.add_edge(i["start"], i["end"], **i)

In [11]:
def get_all_non_company_neighbours(G, origin):
    res = []
    for n in G.neighbors(origin):
        if "Company" not in G.nodes[n]["labels"]:
            res.append(n)
    return res

In [12]:
def create_new_edges(G, origin):
    non_company_neighbours = get_all_non_company_neighbours(G, origin)
    
    for n in non_company_neighbours:
        # get relation type
        r_type = G.edges[origin, n]["type"]
        
        # get all nodes that are connected to it with the same type
        for neighbour in G.neighbors(n):
            # check their type
            if r_type == G.edges[n, neighbour]["type"] and ("Company" in G.nodes[neighbour]["labels"]):
                data = {
                    "start": origin,
                    "end": neighbour,
                    "type": "COMMON_" + r_type
                }
                G.add_edge(origin, neighbour, **data)

In [15]:
nx.number_connected_components(G)

940